In [2]:
from pysgx import stocks, info, util
import importlib
import pandas as pd

In [23]:
importlib.reload(info);

In [16]:
df = pd.DataFrame.from_dict(info.get_active(), orient='index').reset_index()
df.columns = ['Ticker', 'MarketCap']

In [ ]:
df = util.to_df(info.get_all_with_mc(), ['Ticker', 'MarketCap'])
plt = df[df.MarketCap<100000000].MarketCap.plot.hist(bins=50, alpha=0.5, title='SGX Stock Market Cap Distribution - Below(100M SGD)')
plt.set_xlabel("Market Cap (100M)")
plt.get_figure().savefig('mc_100M.png')

In [7]:
df = util.to_df(info.get_all_avg_vol(), ['Ticker', 'AvergeVolume'])
df.AvergeVolume = df.AvergeVolume / 10000

In [ ]:
plt = df[df.AvergeVolume<=1].plot.hist(bins=50, alpha=0.5, title='SGX Average Daily Trading Volume (ADTV) - Below(10K SGD)')
plt.set_xlabel("ADTV (10K)")
plt.legend(['ADTV Total 365 stocks'])
plt.get_figure().savefig('adtv_10K.png')

In [ ]:
plt = df[1<df.AvergeVolume][df.AvergeVolume<=30].plot.hist(bins=50, alpha=0.5, title='SGX Average Daily Trading Volume (ADTV) 10K~300K SGD')
plt.set_xlabel("ADTV (10K)")
plt.legend(['ADTV Total 207 stocks'])
plt.get_figure().savefig('adtv_10K_300K.png')

In [ ]:
plt = df[df.AvergeVolume>30][df.AvergeVolume<=300].plot.hist(bins=50, alpha=0.5, title='SGX Average Daily Trading Volume (ADTV) 300K~3M SGD')
plt.set_xlabel("ADTV (10K)")
plt.legend(['ADTV Total 68 stocks'])
plt.get_figure().savefig('adtv_300K_3M.png')
# len(df[df.AvergeVolume>30][df.AvergeVolume<=300])

In [10]:
df[df.AvergeVolume>300].sort_values(by=['AvergeVolume']).to_csv("adtv_3M_plus.csv", index=False, float_format='%.6f')

In [44]:
dbs = stocks.load("D05")
es3 = stocks.load("ES3")

In [64]:
def get_rclose(stock, start, end):
    df = stock.price[stock.price['Day'] >= start][stock.price['Day'] <= end][["Day", "Close", "Adj Close"]]
    df2 = stock.dividend[stock.dividend["Day"] > "2000-01-01"]
    col = df.apply(lambda row: row.Close + df2[row.Day > df2["Day"]].Dividends.sum(), axis=1)
    df = df.assign(RClose=col.values)
    return df

df = get_rclose(es3, "2010-01-01", "2023-01-01")
# df

C:\Users\wuvis\AppData\Local\Temp\ipykernel_11084\4157375174.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = stock.price[stock.price['Day'] >= start][stock.price['Day'] <= end][["Day", "Close", "Adj Close"]]


In [65]:
df["Adj Close"] = df["Adj Close"] * 990
df["RClose"] = df["RClose"] * 1010
df["Close"] = df["Close"] * 1000

In [ ]:
plt = df.plot(x="Day", y = ["RClose", "Close", "Adj Close"], linewidth=0.8, figsize=(15, 10), title="Esitmated STI")
plt.legend(['Estimated STI if add back dividend', 'STI', 'Estimated STI if use Adj Close'])
plt.get_figure().savefig('STI_estimated.png')